In [1]:
import json
import re
from gensim.models import Word2Vec
import csv
import pandas as pd
from sklearn.cluster import DBSCAN

In [2]:
import snowballstemmer

In [3]:
f = open('digital_trajectoriesraject_public_keyword.json')
digital_trajectoriesraject_public_keyword = json.load(f)

f.close()

In [4]:
f = open('documents1.json')
documents = json.load(f)

f.close()

In [5]:
duplicates_data = pd.read_csv('duplicates_data.csv')
duplicates_data

,0,1,2
0,нереляционными субд,nosql базы данных,1
1,систем управления базами данных,системах управления базами данных,1
2,примере субд,система управления базами данных,1
3,разработке баз данных,реализации бд,1
4,субд,система управления базами данных,1
5,объектно-ориентированным программированием,ооп,1
6,бд,баз данных,1
7,big data,больших данных,1
8,использованием ros,применение пакетов ros,1
9,интеллектуального анализа данных,data mining,1


https://python-school.ru/blog/word2vec-with-examples-in-gensim/

## digital_trajectoriesraject_public_keyword

In [6]:
text = ""
for doc in documents: text += doc['text']

In [7]:
sentences = re.split(r'\.+\s', text.lower())


In [8]:
sentences[:3]

['коммерциализация результатов научных исследований и разработок представляет собой процесс вовлечения их в экономический (коммерческий) оборот в различных сегментах национального и глобального рынков',
 'актуальность данной дисциплины обусловлена становлением на путь модернизации экономики страны',
 'в сложившейся ситуации резко возрастает необходимость оценки и обоснования экономической эффективности проектов, ориентированных на выпуск высокотехнологичной продукции и продвижение новых технологий']

In [9]:
sentences_word_by_word = []
for i in sentences:
    i = re.sub(r'[^а-яёa-z -+.\/0-9]', '', i)
    i = re.sub(r'\xa0', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    sentences_word_by_word.append(i.split(' '))

In [10]:
arr = []
for i in digital_trajectoriesraject_public_keyword:
    i = i['value']
    i = re.sub('[^а-яёa-z -+.\/0-9]', '', i.lower())
    i = re.sub(r'\xa0', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    arr.append(i)
    
digital_trajectoriesraject_public_keyword = list(set([] + arr))

len(digital_trajectoriesraject_public_keyword), digital_trajectoriesraject_public_keyword[:5]

(21727,
 ['рекуррентные сети',
  'завершение процесса формирования',
  'управление рисками ии',
  'системы связи понятие',
  'процессов разработки по'])

In [33]:
w2v_model = Word2Vec(
    min_count=1,
    window=5,
    negative=0,
    alpha=0.1,
    min_alpha=0.005,
    sample=1e-4,
    sg=1,
    hs=-1)

#hs поменял вывод

In [34]:
w2v_model.build_vocab(sentences_word_by_word)

In [35]:
w2v_model.train(sentences_word_by_word, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(81902974, 114943620)

In [36]:
w2v_model.init_sims(replace=True)


/tmp/ipykernel_23917/2897808894.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


In [37]:
#w2v_model.save('w2v_model.model')

In [38]:
#w2v_model = Word2Vec.load('w2v_model.model')

In [39]:
X_test = []
dictionary_test = {}
notInVocab_test = []

for s in duplicates_data.iloc[:,:2].values:
    for phrase in s:
        vec = 0
        flag = 0
        for word in phrase.split(' '):
            if not word in w2v_model.wv.key_to_index.keys():
                flag = 1
                notInVocab_test.append(word)
                break
            vec += w2v_model.wv[word]
        vec /= len(phrase.split(' '))
        if flag:
            continue
        string = " ".join(str(v) for v in vec)
        if string in dictionary_test.keys():
            dictionary_test[string].append(phrase)
        else:
            dictionary_test[string] = [phrase]
        X_test.append(vec)
    
X_test[0]

array([-1.36928242e-02,  5.14534861e-02, -1.05764009e-02, -1.20137103e-01,
       -4.15516123e-02,  1.47973061e-01, -7.93567002e-02,  4.61596772e-02,
       -4.65071723e-02,  2.30911970e-02,  6.98951334e-02, -2.86934357e-02,
        4.16838825e-02, -3.00431289e-02,  9.73434001e-02, -3.74228470e-02,
       -1.07600223e-02, -2.07814239e-02, -8.23886544e-02,  8.04062933e-02,
       -6.57801926e-02,  5.51556535e-02,  1.10010117e-01,  5.64829893e-02,
       -7.95783550e-02,  8.40503722e-02, -9.58048403e-02, -5.73639087e-02,
        5.01477625e-03,  1.80199683e-01, -6.73692822e-02,  1.57362029e-01,
        5.48116677e-02, -1.46612525e-01,  5.48373871e-02,  6.39323369e-02,
        6.20282814e-03,  6.18616864e-02, -1.16919681e-01,  3.28303128e-02,
        1.65957138e-02, -6.02414049e-02,  1.26539707e-01, -1.13397174e-01,
       -1.78777389e-02,  2.43777316e-02, -3.79481614e-02,  1.13015488e-01,
        2.64290515e-02,  1.07137777e-01, -3.56326178e-02, -7.12797884e-03,
        2.47175638e-02, -

In [40]:
import numpy as np
data_len = len(X_test) // 2
arr1 = []
arr0 = []
for i in range(data_len):
    point_1 = X_test[i * 2]
    point_2 = X_test[i * 2 + 1]
    distance = np.sqrt(sum((a-b) ** 2 for a, b in zip(point_1, point_2)))
    if duplicates_data.iloc[i, 2] == 1:
        arr1.append(distance)
    else:
        arr0.append(distance)


In [41]:
min1 = round(min(arr1), 4)
max1 = round(max(arr1), 4)
mean1 = round(np.array(arr1).mean(), 4)
med1 = round(np.median(np.array(arr1)), 4)

print('min =', min1, 
      'max =', max1, 
      'mean =', mean1,
      'median =', med1)

arr1, sorted(arr1)


min = 0.2088 max = 1.2461 mean = 0.9087 median = 0.9382


([0.8641527750703196,
  0.20875379389820892,
  0.8189384389805104,
  0.6794438463156017,
  0.9749111136397136,
  1.2225446493112397,
  1.0000978929534297,
  1.0287719112203342,
  0.9013895740864396,
  1.118209605734147,
  1.246057358208581,
  0.80720256698686,
  1.030268170889557,
  0.8213124969838195],
 [0.20875379389820892,
  0.6794438463156017,
  0.80720256698686,
  0.8189384389805104,
  0.8213124969838195,
  0.8641527750703196,
  0.9013895740864396,
  0.9749111136397136,
  1.0000978929534297,
  1.0287719112203342,
  1.030268170889557,
  1.118209605734147,
  1.2225446493112397,
  1.246057358208581])

In [42]:
min0 = round(min(arr0), 4)
max0 = round(max(arr0), 4)
mean0 = round(np.array(arr0).mean(), 4)
med0 = round(np.median(np.array(arr0)), 4)

print('min =', min0, 
      'max =', max0, 
      'mean =', mean0,
      'median =', med0)

arr0, sorted(arr0)

min = 0.6804 max = 1.238 mean = 0.9358 median = 0.9509


([0.9509059188525486,
  0.9390046658054302,
  0.695102533069674,
  0.6803980510677966,
  1.034296121076799,
  0.8768797786085482,
  0.7502631033588022,
  0.8612694533719151,
  1.033388970685699,
  1.020467875391314,
  1.1108202988587186,
  0.9601523338835725,
  1.0125256220950287,
  1.2379677746385067,
  0.8741357511116239],
 [0.6803980510677966,
  0.695102533069674,
  0.7502631033588022,
  0.8612694533719151,
  0.8741357511116239,
  0.8768797786085482,
  0.9390046658054302,
  0.9509059188525486,
  0.9601523338835725,
  1.0125256220950287,
  1.020467875391314,
  1.033388970685699,
  1.034296121076799,
  1.1108202988587186,
  1.2379677746385067])

In [43]:
print('min0-min1=', round(min0 - min1, 4), 
      '\nmax0-max1=', round(max0 - max1, 4), 
      '\nmean0-mean1=', round(mean0 - mean1, 4), 
      '\nmedian0-median1=', round(med0 - med1, 4))

min0-min1= 0.4716 
max0-max1= -0.0081 
mean0-mean1= 0.0271 
median0-median1= 0.0127


## При вычислении среднего из векторов

### 1


## При суммировании векторов

### Вариант n
при 

    min_count=1,
    window=5,
    negative=0,
    alpha=0.7,
    min_alpha=0.01,
    sample=1e-1,
    sg=1,
    hs=1
    
w2v_model.train(sentences_word_by_word, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

для 1: min = 0.2162 max = 3.7256 mean = 1.9104 median = 1.9625

для 0: min = 0.9745 max = 3.6274 mean = 2.0252 median = 2.1101

**0.2162** бд - баз данных

**3.7256** нереляционными субд	nosql базы данных	
 
 
**0.9745** оценку структуры	другие методы

**3.6274** систем моделирования	методы исследования свойств клеток


### Вариант №m

min0-min1= 0.3275

max0-max1= 0.6789 

mean0-mean1= 0.3233 

median0-median1= 0.1946

w2v_model = Word2Vec(

    min_count=1,
    window=5,
    negative=0,
    alpha=0.5,
    min_alpha=0.005,
    sample=1e-4,
    sg=1,
    hs=1)
    
w2v_model.train(sentences_word_by_word,total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

1: min = 0.6212 max = 2.8116 mean = 1.7783 median = 1.7289

0: min = 0.9487 max = 3.4905 mean = 2.1016 median = 1.9235

In [44]:
X = []
dictionary = {}
notInVocab = []

for phrase in digital_trajectoriesraject_public_keyword:
    vec = 0
    flag = 0
    #print(phrase)
    for word in phrase.split(' '):
        if not word in w2v_model.wv.key_to_index.keys():
            flag = 1
            notInVocab.append(word)
            break
        vec += w2v_model.wv[word]
    vec /= len(phrase.split(' '))
    if flag:
        continue
    string = " ".join(str(v) for v in vec)
    if string in dictionary.keys():
        dictionary[string].append(phrase)
    else:
        dictionary[string] = [phrase]
    X.append(vec)
    
X[0]

array([-0.16103783, -0.1044266 ,  0.08361261,  0.02350327, -0.2527531 ,
       -0.06969027, -0.08781861, -0.16811746,  0.04115742, -0.0110709 ,
       -0.03808708, -0.15999746, -0.03492431,  0.02105848,  0.06554188,
       -0.1527342 ,  0.01083815,  0.03649223,  0.12581836, -0.06109098,
       -0.01461799,  0.04414637,  0.13672377,  0.00548862,  0.02335389,
       -0.02964743, -0.04802755,  0.03269113, -0.10658562,  0.00060132,
       -0.02508644, -0.01939202,  0.16676567, -0.12359335,  0.16784126,
        0.05993452,  0.03742409, -0.07207196, -0.07653012, -0.07580516,
       -0.0787745 ,  0.08485495, -0.02992948,  0.00177709,  0.10609607,
        0.06737754,  0.07887678,  0.01685133,  0.02852515,  0.04503139,
       -0.04131873,  0.07213522,  0.05749729,  0.03015897, -0.07405467,
        0.11906563, -0.16481535,  0.06975165,  0.02193389,  0.052607  ,
        0.02355509,  0.00444989, -0.03222206, -0.16124639,  0.13114311,
       -0.08052769,  0.04404899,  0.06801581, -0.16467789, -0.00

### DBSCAN

In [45]:
clustering = DBSCAN(eps=0.05, min_samples=2, metric='cosine').fit(X)


In [46]:
clustering.labels_

array([ 0, -1, -1, ..., -1, -1, -1])

In [47]:
for i in range(-1, 20):
    print(list(clustering.labels_).count(i))

20692
2
2
2
2
2
6
5
3
2
24
5
3
2
3
2
3
2
2
2
2


In [48]:
number_of_clusters = clustering.labels_.max()
synonyms_array = []
size = len(X)
labels = list(clustering.labels_)

for i in range(number_of_clusters):
    cluster = np.where(clustering.labels_ == i)[0]
    synonyms = []
    for vec in cluster:
        synonyms.extend(dictionary[" ".join([str(v) for v in X[vec]])])
    synonyms_array.append(synonyms)
            

In [49]:
for i in range(len(synonyms_array)):
    print(i)
    for j in synonyms_array[i][:20]:
        print(j)
    if len(synonyms_array[i]) > 20:
        print('...')
    print()

0
рекуррентные сети
рекуррентные нейронные сети

1
навыков построения алгоритмов обработки сигналов
навыков синтеза алгоритмов обработки сигналов

2
рентгеновского излучения
излучения рентгеновского
рентгеновского излучения
излучения рентгеновского

3
технологий решения
технологий решения задач

4
последовательностных устройств
последовательностных цифровых устройств

5
умений анализа
навыков анализа
освоение навыков анализа
приобретение навыков анализа
формирование умений анализа
формирование навыков анализа

6
области баз данных
навыков проектирования баз данных
формирование навыков проектирования баз данных
области проектирования баз данных
формирование умений разработки баз данных

7
цель дисциплины изучение
цель освоения дисциплины изучение методов
цель дисциплины изучение принципов

8
самостоятельного решения задач
самостоятельного решения

9
задач моделирования
освоение навыков постановки задач
формирование навыков решения задач
навыков решения задач моделирования
формирование н

In [50]:
for i in list(dictionary.keys()):
    if len(dictionary[i]) > 1:
        print(dictionary[i], '\n')

['рентгеновского излучения', 'излучения рентгеновского'] 

['свч устройств', 'устройств свч'] 

['источника напряжения', 'напряжения источника'] 

['ответа варианты', 'варианты ответа'] 

['tcad synopsys', 'synopsys tcad'] 

['систем информационных', 'информационных систем'] 

['проектирования средств', 'средств проектирования'] 

['информационных компонентов', 'компонентов информационных'] 

['навыков работы', 'работы навыков'] 

['проведения самостоятельного', 'самостоятельного проведения'] 

['оптического излучения', 'излучения оптического'] 

['моделирования систем', 'систем моделирования'] 

['систем поддержки', 'поддержки систем'] 

['алгоритмов оптимизации', 'оптимизации алгоритмов'] 

['методов организации', 'организации методов'] 

['различных лазеров', 'лазеров различных'] 

['свч фильтры', 'фильтры свч'] 

['применения методов', 'методов применения'] 

['классификации систем', 'систем классификации'] 

['представления моделей', 'моделей представления'] 

['проектирования объ